In [1]:
import numpy as np 
import rospy 
import sys
sys.path.append("..")
from kinematics.utils.util_ik import make_ik_input
from kinematics.class_structure import CHAIN
from kinematics.utils.util_ik import add_joints, find_route
print("Done.")

Done.


In [2]:
class RobotClass(object):
    def __init__(self, file_name, base_offset):
        super(RobotClass,self).__init__() 
        self.chain = CHAIN(file_name=file_name, base_offset=base_offset)
        self.chain.add_joi_to_robot()
        self.chain.add_link_to_robot()
        self.chain.fk_chain(1)
    
    def solve_ik(self,  target_name      = ['wrist_3_joint'],
                        target_position  = [[0,0,0]],
                        target_rotation  = [[-1.57, 0, -1.57]],
                        solve_position   = [1],
                        solve_rotation   = [1],
                        weight_position  = 1,
                        weight_rotation  = 1,
                        joi_ctrl_num= 6): 
                        
        ingredients = make_ik_input(target_name = target_name,
                                    target_pos  = target_position,
                                    target_rot  = target_rotation,
                                    solve_pos   = solve_position,
                                    solve_rot   = solve_rotation,
                                    weight_pos  = weight_position,
                                    weight_rot  = weight_rotation,
                                    joi_ctrl_num= joi_ctrl_num)
        total_q = self.chain.get_q_from_ik(ingredients)
        ctrl_q  = total_q[:joi_ctrl_num+1] #Including base joint for Unity Env
        ctrl_q  = ctrl_q.reshape(-1,)
        return ctrl_q 

### IK Solve: Panda

In [4]:
file_name = "../urdf/panda/panda.urdf"
base_offset = [0,0,0]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
q         = robot.solve_ik(target_name    = ['panda_joint7'],
            target_position  = [[0.,0,0.8]],
            target_rotation  = [[0, 0, 0]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 7)
print("Control joint value: {} \nShape: {}".format(q, q.shape))
for i in range(8):
    print(robot.chain.joint[i].name)
    print(robot.chain.joint[i].p)

Control joint value: [ 0.00000000e+00  1.83653589e+00  8.90053737e-01  4.18356937e-07
  1.24100892e+00 -8.62165418e-07  2.79063523e+00  1.30505775e+00] 
Shape: (8,)
base_joint
[[0]
 [0]
 [0]]
panda_joint1
[[0.   ]
 [0.   ]
 [0.333]]
panda_joint2
[[0.   ]
 [0.   ]
 [0.333]]
panda_joint3
[[-0.0644911 ]
 [ 0.23694566]
 [ 0.531881  ]]
panda_joint4
[[-0.07812732]
 [ 0.28704612]
 [ 0.4677698 ]]
panda_joint5
[[-0.02311069]
 [ 0.08491107]
 [ 0.7999999 ]]
panda_joint6
[[-0.02311069]
 [ 0.08491107]
 [ 0.7999999 ]]
panda_joint7
[[ 8.67919643e-08]
 [-1.20457097e-08]
 [ 8.00000099e-01]]


### IK Solve: UR5e

In [3]:
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0,0,0]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
q         = robot.solve_ik(target_name = ['wrist_3_joint'],
            target_position  = [[0.2,0,0.6]],
            target_rotation  = [[0, 0, 0]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 6)

print("Control joint value: {} \nShape: {}".format(q, q.shape))
for i in range(15):
    print(robot.chain.joint[i].name)
    print(robot.chain.joint[i].p)

Control joint value: [ 0.         -0.73420691 -2.0789368   1.64251504  0.43642175 -0.73420689
  3.14159114] 
Shape: (7,)
base_joint
[[0]
 [0]
 [0]]
shoulder_pan_joint
[[0.   ]
 [0.   ]
 [0.163]]
shoulder_lift_joint
[[0.09245981]
 [0.10244601]
 [0.163     ]]
elbow_joint
[[-0.14881959]
 [ 0.14374236]
 [ 0.53430158]]
wrist_1_joint
[[ 0.11491045]
 [-0.09427989]
 [ 0.69999972]]
wrist_2_joint
[[2.00000273e-01]
 [1.28983631e-07]
 [6.99999718e-01]]
wrist_3_joint
[[2.00000275e-01]
 [1.27713470e-07]
 [5.99999718e-01]]
ee_fixed_joint
[[0.20000028]
 [0.10000013]
 [0.59999972]]
gripper_base_joint
[[0.20000028]
 [0.10000013]
 [0.59999972]]
gripper_tcp_joint
[[0.20000028]
 [0.27000007]
 [0.60013509]]
gripper_finger1_joint
[[0.21690026]
 [0.20499189]
 [0.61038335]]
gripper_finger1_inner_knuckle_joint
[[0.20740026]
 [0.22149213]
 [0.61009648]]
gripper_finger1_finger_tip_joint
[[0.20940027]
 [0.27645945]
 [0.60594025]]
gripper_finger2_joint
[[0.18290026]
 [0.20499189]
 [0.6103833 ]]
gripper_finger2_inne

In [5]:
import math 
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0,0,0]
joints      = np.array([0, -math.pi/2, math.pi/2, -math.pi/2, -math.pi/2, 0])
robot       = RobotClass(file_name=file_name, base_offset=base_offset)
# Index From Root 
idx_fr_root = find_route(robot.chain.joint, 7) # base joint + 6-DoF
print("Idx from root: {}".format(idx_fr_root))
add_joints(robot.chain.joint, idx_fr_root,joints)
robot.chain.fk_chain(1)
for i in range(8):
    print(robot.chain.joint[i].name)
    print(robot.chain.joint[i].p.T)


Idx from root: [2 3 4 5 6 7]
base_joint
[[0 0 0]]
shoulder_pan_joint
[[0.    0.    0.163]]
shoulder_lift_joint
[[0.    0.138 0.163]]
elbow_joint
[[-2.08100308e-12  7.00000000e-03  5.88000000e-01]]
wrist_1_joint
[[0.392 0.007 0.588]]
wrist_2_joint
[[0.392 0.134 0.588]]
wrist_3_joint
[[0.492 0.134 0.588]]
ee_fixed_joint
[[0.492 0.134 0.488]]


In [11]:
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0,0,0]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
q         = robot.solve_ik(target_name = ['wrist_3_joint'],
            target_position  = [[0.492,0.134,0.588]],
            target_rotation  = [[-math.pi/2, 0, -math.pi/2]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 6)
print("Result: {}".format(q[1:])) # Without base joint  
print("Goal: {}".format(np.array([0, -math.pi/2, math.pi/2, -math.pi/2, -math.pi/2, 0])))

Result: [ 9.48379523e-09 -1.57079596e+00  1.57079664e+00 -1.57079612e+00
 -1.57079542e+00  8.84868418e-07]
Goal: [ 0.         -1.57079633  1.57079633 -1.57079633 -1.57079633  0.        ]
